In [ ]:
import pandas as pd
import numpy as np

In [2]:
prices = [50, 100, 120, 100, 100, 90, 110, 140]

In [ ]:
df_prices = pd.DataFrame({"time": np.arange(len(prices)), "price": prices})
df_prices

,time,price
0,0,50
1,1,100
2,2,120
3,3,100
4,4,100
5,5,90
6,6,110
7,7,140


In [ ]:
df_prices["MA"] = df_prices["price"].rolling(window=2).mean()
df_prices

,time,price,MA
0,0,50,NaN
1,1,100,75.0
2,2,120,110.0
3,3,100,110.0
4,4,100,100.0
5,5,90,95.0
6,6,110,100.0
7,7,140,125.0


In [ ]:
def do_trade(row, prev_row, amount=200, max_percent=0.5):
    if row["price"] > row["MA"]:
        return int(amount / row["price"])
    elif row["price"] < row["MA"]:
        return -int(prev_row["hold_Q"] * max_percent)
    else:
        return 0

In [ ]:
df_prices["trade_Q"] = 0
df_prices["hold_Q"] = 0
for index, row in df_prices.iterrows():
    if index == 0:
        continue
    elif index >= 7:
        break
    else:
        prev_row = df_prices.loc[index - 1]
    df_prices.loc[index, "trade_Q"] = do_trade(row, prev_row)
    df_prices.loc[index, "hold_Q"] = (
        prev_row["hold_Q"] + df_prices.loc[index, "trade_Q"]
    )

df_prices.loc[index:, "hold_Q"] = df_prices.loc[index - 1, "hold_Q"]
df_prices

,time,price,MA,trade_Q,hold_Q
0,0,50,NaN,0,0
1,1,100,75.0,2,2
2,2,120,110.0,1,3
3,3,100,110.0,-1,2
4,4,100,100.0,0,2
5,5,90,95.0,-1,1
6,6,110,100.0,1,2
7,7,140,125.0,0,2


In [ ]:
def compute_PL(df_prices, idx):
    sub_df = df_prices.loc[:idx]
    net_inflow = -(sub_df["price"] * sub_df["trade_Q"]).sum()
    close_inflow = sub_df.loc[idx, "hold_Q"] * sub_df.loc[idx, "price"]
    return net_inflow + close_inflow


for idx in range(len(df_prices)):
    df_prices.loc[idx, "PL"] = compute_PL(df_prices, idx)

df_prices

,time,price,MA,trade_Q,hold_Q,PL
0,0,50,NaN,0,0,0.0
1,1,100,75.0,2,2,0.0
2,2,120,110.0,1,3,40.0
3,3,100,110.0,-1,2,-20.0
4,4,100,100.0,0,2,-20.0
5,5,90,95.0,-1,1,-40.0
6,6,110,100.0,1,2,-20.0
7,7,140,125.0,0,2,40.0
